## Making csv file

In [52]:
import codecs
from bs4 import BeautifulSoup
import csv
import os

In [62]:
csv_file.close()

In [63]:
csv_file = open('dou_vacancies.csv', 'w', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['headline', 'description'])

22

In [64]:
directory = 'dou/rezult'
for filename in os.listdir(directory):    
    file = codecs.open(directory + '/' + filename, "r", 'utf-8')
    soup = BeautifulSoup(file)
    headline = soup.find('h1', class_="g-h2").get_text()
    vacancy = soup.find('div', class_="l-vacancy")
    descriptions = vacancy.find_all('div', class_="text b-typo vacancy-section")
    text = ''
    for description in descriptions:
        text += description.get_text()
    csv_writer.writerow([headline, text])

In [ ]:
import pandas as pd

In [65]:
df = pd.read_csv('dou_vacancies.csv')

In [67]:
df.head()

,headline,description
0,"Junior ML engineer (Computer Vision, 3d)",\r\n— Python— хорошая математическая база для ...
1,Senior Java/BigData Engineer,\r\nExperience in Java development (5 years+);...
2,Java Engineer for HighLoad/BigData project | SM,\r\n— Professional Java experience 3+ years— O...
3,NLP Engineer,"\r\n— 2+ years experience with Python, includi..."
4,Data Engineer,\r\n— 2+ years of hands-on experience as a Dat...


## Text processing

In [103]:
import re
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from string import punctuation
punctuation += '—'

In [90]:
stop = set(stopwords.words('english'))
punct = list(punctuation)

In [83]:
def clean_text(text):
    text = text.replace(u'\xa0', u' ')
    text_eng = re.sub('[^a-zA-Z0-9-+ ]+', '', text)
    return text_eng

In [92]:
def remove_stop_words(text):
    lst = word_tokenize(text)
    lst = [word for word in lst if word not in stop and word not in punct]
    return lst

In [123]:
def leave_nouns(lst):
    new_lst = [word for (word,pos) in nltk.pos_tag(lst) if pos[:2] == 'NN']
    return new_lst

In [99]:
df['description'] = df.description.apply(clean_text)

In [113]:
df['description'] = df.description.apply(remove_stop_words)

In [125]:
df['description'] = df.description.apply(leave_nouns)

In [119]:
df['is_data_engineer'] = df['headline'].apply(lambda x: 'data' in x.lower())

In [132]:
df.head()

,headline,description,is_data_engineer
0,"Junior ML engineer (Computer Vision, 3d)","[Python, machine, learning, Numpy, Pandas, Sci...",False
1,Senior Java/BigData Engineer,"[Experience, Java, development, years+Experien...",True
2,Java Engineer for HighLoad/BigData project | SM,"[Java, experience, years, OOPdesign, proficien...",True
3,NLP Engineer,"[years, Python, engineering, experience, API, ...",False
4,Data Engineer,"[years, experience, Data, Engineer, experience...",True


In [133]:
df['is_data_engineer'].value_counts()

True     118
False     38
Name: is_data_engineer, dtype: int64

In [139]:
df_engineers = df.query('is_data_engineer == True').copy()

In [140]:
df_engineers['description']

1      [Experience, Java, development, years+Experien...
2      [Java, experience, years, OOPdesign, proficien...
4      [years, experience, Data, Engineer, experience...
5      [Clients, mission, everyone, eat, Company, mea...
6      [years, Scala, development, cloud, services, A...
                             ...                        
151    [Experience, data, analytics, data, engineerin...
152    [cross-platformcross-browser, QA, solution, do...
153    [Fractal, Kyiv, team, position, junior, data, ...
154    [years, experience, software, development, pri...
155    [Job, SummaryDevelop, Rokus, edge, delivery, i...
Name: description, Length: 118, dtype: object

In [141]:
df_engineers['text'] = df_engineers['description'].apply(lambda x : " ".join(x))

In [144]:
all_text = df_engineers['text'].str.cat(sep=', ')

In [149]:
import spacy
from spacy.lang.en import English
nlp = spacy.load('en_core_web_md')

In [150]:
doc = nlp(all_text)

In [154]:
list_technologies = []

In [155]:
for ent in doc.ents:
    if ent.label_ != 'DATE':
        list_technologies.append(ent.text)

In [158]:
set_of_technologies = set(list_technologies)